<a href="https://colab.research.google.com/github/edwin2017/BotOrderShop/blob/master/%D0%A0%D0%B0%D0%B7%D0%B1%D0%BE%D1%80_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F_Python_Bootcamp_Day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание

Улучшите своего бота, добавив функциональность сжатия изображения

Информация об изображении хранится в атрибуте `size`, который имеет тип tuple (кортеж).

С кортежами работают так же, как со списками: в квадратных скобках - индекс элемента. Нумерация начинается с 0.


Изменение разрешения производится через функцию .resize((ширина, высота)) - аргументом передается кортеж, который сам записывается в круглых скобках

In [ ]:
# width = source_image.size[0]
# height = source_image.size[1]

Арифметические операции в Python:
* `a + b` — плюс
* `a - b` — минус
* `a * b` — умножить
* `a / b` — поделить
* `a // b` — поделить нацело, отбросив остаток
* `a % b` — получить остаток от деления
* `round(a)` — округлить

In [ ]:
!pip install pyTelegramBotAPI SpeechRecognition pydub

In [ ]:
import os
import telebot
import speech_recognition
from pydub import AudioSegment
from PIL import Image, ImageEnhance, ImageFilter


# ↓↓↓ Ниже нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '2007628239:AAEF4ZVqLiRKG7j49EC4vaRwXjJ6DN6xng8'
token = '<ваш токен>'  # <<< Ваш токен

bot = telebot.TeleBot(token)


def transform_image(filename):
    # Функция обработки изображения
    source_image = Image.open(filename)
    enhanced_image = source_image.filter(ImageFilter.EMBOSS)
    enhanced_image = enhanced_image.convert('RGB')
    width = enhanced_image.size[0]
    height = enhanced_image.size[1]

    enhanced_image = enhanced_image.resize((width // 2, height // 2))

    enhanced_image.save(filename)
    return filename


@bot.message_handler(content_types=['photo'])
def resend_photo(message):
    # Функция отправки обработанного изображения
    file_id = message.photo[-1].file_id
    filename = download_file(bot, file_id)

    # Трансформируем изображение
    transform_image(filename)

    image = open(filename, 'rb')
    bot.send_photo(message.chat.id, image)
    image.close()
    
    # Не забываем удалять ненужные изображения
    if os.path.exists(filename):
        os.remove(filename)


def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename


def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:     
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text


def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename


@bot.message_handler(commands=['start'])
def say_hi(message):
    # Функция, отправляющая "Привет" в ответ на команду /start
    bot.send_message(message.chat.id, 'Привет')


@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    bot.send_message(message.chat.id, text)


# Запускаем бота. Он будет работать до тех пор, пока работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()